In [ ]:
import numpy as np
import torch
import os
import pickle

from Models import build
from AllSketchGenerator import time_all_sketches_all_graphs,generate_ErdosRenyi_randomSeeds

model_num = 1
training_switch = -1
k_max = 4
cuda = 0

if model_num == 1:
    model_name = '128 64 32 16'
    hidden_widths = [128,64,32,16]
elif model_num == 2:
    model_name = '64 32 16 8'
    hidden_widths = [64,32,16,8]
elif model_num == 3:
    model_name = '32 16 8 4'
    hidden_widths = [32,16,8,4]
elif model_num == 4:
    model_name = '128 64 32'
    hidden_widths = [128,64]
elif model_num == 5:
    model_name = '64 32 16'
    hidden_widths = [64,32,16]
elif model_num == 6:
    model_name = '32 16 8'
    hidden_widths = [32,16,8]
elif model_num == 7:
    model_name = '128 64'
    hidden_widths = [128,64]
elif model_num == 8:
    model_name = '64 32'
    hidden_widths = [64,32]
elif model_num == 9:
    model_name = '32 16'
    hidden_widths = [32,16]
elif model_num == 0:
    model_name = 'test'
    hidden_widths = [16]
else:
    raise AssertionError('Model not specified.')

if model_num == 1:
    GNN_only = False
else:
    GNN_only = True

if training_switch == -1:
    output_dir = 'unmasked'
elif training_switch == 0:
    output_dir = 'switch0'
elif training_switch == 1:
    output_dir = 'switch1'
elif training_switch == 2:
    output_dir = 'switch2'
elif training_switch == 3:
    output_dir = 'switch3'
elif training_switch == 4:
    output_dir = 'switch4'
elif training_switch == 5:
    output_dir = 'switch5'
elif training_switch == 6:
    output_dir = 'switch6'
elif training_switch == 7:
    output_dir = 'switch7'
elif training_switch == 8:
    output_dir = 'switch8'
elif training_switch == 9:
    output_dir = 'switch9'
elif training_switch == 10:
    output_dir = 'switch10'
elif training_switch == 11:
    output_dir = 'switch11'
elif training_switch == 12:
    output_dir = 'switch12'
elif training_switch == 13:
    output_dir = 'switch13'
elif training_switch == 14:
    output_dir = 'switch14'
else:
    raise AssertionError('Invalid training switch.')

graph_type = 'ErdosRenyi'
sample_dir = os.path.dirname(os.getcwd())+'/samples/'+graph_type+'_lambda_div_n'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

all_out_dir = sample_dir.replace('samples','outputs')
if not os.path.exists(all_out_dir):
    os.makedirs(all_out_dir)
output_dir = all_out_dir + '/' + output_dir

model_dir = output_dir + '/' + model_name
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
for n in [25]+list(2**np.array(range(7))*50):
    for lbd in [6,5,4,3]:
        
        title = 'n = '+str(n)+', lambda = '+str(lbd)
        print(title)
        dir = model_dir+'/lbd '+str(lbd)
        if not os.path.exists(dir):
            os.makedirs(dir)

        if model_num == 1:
            flag = False
            networkx_dur_path = dir+'/networkx_dur_metric=random_'+str(n)+'.pkl'
            if not os.path.exists(networkx_dur_path):
                flag = True
            if flag:
                p = sample_dir+'/'+graph_type+'_sketch_'+str(n)+'_'+str(lbd)+'.pkl'
                if os.path.exists(p):
                    with open(p, 'rb') as file:
                        samples = pickle.load(file)
                dur_data = time_all_sketches_all_graphs(samples,cuda,criterion_type='mse',method='networkx')
                with open(networkx_dur_path, 'wb') as file:
                    pickle.dump(dur_data, file)
        
        flag = False
        dur_path = dir+'/GNN_dur_metric=random_'+str(n)+'.pkl'
        if not os.path.exists(dur_path):
            flag = True
            dur_data = [False,False,False,False], [None,None,None,None]
        else:
            with open(dur_path, 'rb') as file:
                dur_data = pickle.load(file)
            if False in dur_data[0]:
                flag == True
    
        if flag:

            done = dur_data[0]
            dur_list = dur_data[1]
            out = int(np.floor(np.sqrt(n)))

            # p = sample_dir+'/'+graph_type+'_training_'+str(n)+'_'+str(lbd)+'.pkl'
            # if os.path.exists(p):
            #     with open(p, 'rb') as file:
            #         samples = pickle.load(file)
            # else:
            #     samples = generateSamples_training_ErdosRenyi(sample_dir,title,200,50,50,n,lbd)
            #     with open(p, 'wb') as file:
            #         pickle.dump(samples, file)
            
            if not done[0]:
                model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GCN','mse','adam','cyclic-cosine','reLU')
                file = model_dir+'/'+model.name+'_out'+str(out)+'.pth'
                if os.path.exists(file):
                    model.load_state_dict(torch.load(file))
                print(model)
                model1 = model
                p = sample_dir+'/'+graph_type+'_sketch_'+str(n)+'_'+str(lbd)+'.pkl'
                if os.path.exists(p):
                    with open(p, 'rb') as file:
                        samples = pickle.load(file)
                else:
                    samples = generate_ErdosRenyi_randomSeeds(10,4,n,lbd)
                    with open(p, 'wb') as file:
                        pickle.dump(samples, file)
                dur_list[0] = time_all_sketches_all_graphs(samples,cuda,model1=model1,criterion_type='mse',method='GNN')
                done[0] = True
                dur_data = done,dur_list
                with open(dur_path, 'wb') as file:
                    pickle.dump(dur_data, file)
            if not done[1]:
                model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'SAGE','mse','adam','cyclic-cosine','reLU')
                file = model_dir+'/'+model.name+'_out'+str(out)+'.pth'
                if os.path.exists(file):
                    model.load_state_dict(torch.load(file))
                print(model)
                model2 = model
                p = sample_dir+'/'+graph_type+'_sketch_'+str(n)+'_'+str(lbd)+'.pkl'
                if os.path.exists(p):
                    with open(p, 'rb') as file:
                        samples = pickle.load(file)
                else:
                    samples = generate_ErdosRenyi_randomSeeds(10,4,n,lbd)
                    with open(p, 'wb') as file:
                        pickle.dump(samples, file)
                dur_list[1] = time_all_sketches_all_graphs(samples,cuda,model1=model2,criterion_type='mse',method='GNN')
                done[1] = True
                dur_data = done,dur_list
                with open(dur_path, 'wb') as file:
                    pickle.dump(dur_data, file)
            if not done[2]:
                model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GAT','mse','adam','cyclic-cosine','reLU')
                file = model_dir+'/'+model.name+'_out'+str(out)+'.pth'
                if os.path.exists(file):
                    model.load_state_dict(torch.load(file))
                print(model)
                model3 = model
                p = sample_dir+'/'+graph_type+'_sketch_'+str(n)+'_'+str(lbd)+'.pkl'
                if os.path.exists(p):
                    with open(p, 'rb') as file:
                        samples = pickle.load(file)
                else:
                    samples = generate_ErdosRenyi_randomSeeds(10,4,n,lbd)
                    with open(p, 'wb') as file:
                        pickle.dump(samples, file)
                dur_list[2] = time_all_sketches_all_graphs(samples,cuda,model1=model3,criterion_type='mse',method='GNN')
                done[2] = True
                dur_data = done,dur_list
                with open(dur_path, 'wb') as file:
                    pickle.dump(dur_data, file)
            if not done[3]:
                model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GIN','mse','adam','cyclic-cosine','reLU')
                file = model_dir+'/'+model.name+'_out'+str(out)+'.pth'
                if os.path.exists(file):
                    model.load_state_dict(torch.load(file))
                print(model)
                model4 = model
                p = sample_dir+'/'+graph_type+'_sketch_'+str(n)+'_'+str(lbd)+'.pkl'
                if os.path.exists(p):
                    with open(p, 'rb') as file:
                        samples = pickle.load(file)
                else:
                    samples = generate_ErdosRenyi_randomSeeds(10,4,n,lbd)
                    with open(p, 'wb') as file:
                        pickle.dump(samples, file)
                dur_list[3] = time_all_sketches_all_graphs(samples,cuda,model1=model4,criterion_type='mse',method='GNN')
                done[3] = True
                dur_data = done,dur_list
                with open(dur_path, 'wb') as file:
                    pickle.dump(dur_data, file)